In [2]:
import sys

# So it can see the scripts folder
sys.path.append("../")

In [3]:
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.llms.openai import OpenAI
from scripts.constants import GPT3_MODEL
from scripts.parse_json import PDFJson
from llama_index.core import Document
import nest_asyncio
nest_asyncio.apply()


/Users/rashad/Library/Caches/pypoetry/virtualenvs/rag-project-RXD1v8WZ-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
node_parser = MarkdownElementNodeParser(
    llm=OpenAI(model=GPT3_MODEL), num_workers=8
)
pdf_json = PDFJson.from_json_file("../parse-output/2302.01381.json")
document = Document(metadata={"pdf_path": pdf_json.file_path, "pdf_id": pdf_json.job_id}, text=pdf_json.full_md)

In [6]:
nodes = node_parser.get_nodes_from_documents([document])
nodes



10it [00:00, 37718.56it/s]
  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:04<00:00,  2.26it/s]


[TextNode(id_='29c4684f-4fee-4e11-8d86-32847f1a9a08', embedding=None, metadata={'pdf_path': './corpus/2302.01381.pdf', 'pdf_id': 'ef725e87-9f67-4a94-859d-ec9285017cf0'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='cd7e92c1-330d-43bc-af36-0c9e26a2ec76', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='20b357706ccba394bb1a1199bf2f75fd36aafd245d0c7dadd0fe2e1e68c2199b'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='87e48e4d-fe81-4a00-b441-5d70584bbfbc', node_type=<ObjectType.TEXT: '1'>, metadata={'pdf_path': './corpus/2302.01381.pdf', 'pdf_id': 'ef725e87-9f67-4a94-859d-ec9285017cf0'}, hash='a1c1425721076dfc42ba055a4bd9f2632f99b27236bc562f5517754c7f383b5e')}, text='Effective Robustness against Natural Distribution Shifts\n\n Effective Robustness against Natural Distribution Shifts for Models with Different Training Data\n\nZhouxing Shi* &emsp; Nicholas Carlini &emsp; Ananth Balashan

In [7]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

In [8]:
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import StorageContext
from qdrant_client import QdrantClient

client = QdrantClient(path="../qdrant-vector-store")
vector_store = QdrantVectorStore(client=client, collection_name="test-collection")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [16]:
index = VectorStoreIndex(nodes=base_nodes, storage_context=storage_context)
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

# reranker = FlagEmbeddingReranker(
#     top_n=5,
#     model="BAAI/bge-reranker-large",
# )
query_engine = index.as_query_engine(similarity_top_k=10, verbose=True)



In [18]:
question = (
    "What is the role of ImageNet-R as the OOD test set in the evaluation process?	"
)
response = query_engine.query(question)



In [23]:
response.response

'ImageNet-R serves as an out-of-distribution (OOD) test set in the evaluation process.'

In [25]:
for source in response.source_nodes:
    print(source.text)


We also train models by combining ImageNet and YFCC-15M and LAION-15M, respectively. We discard models with ImageNet accuracy below 5%. Additionally, in Section 5.4, we also have downloaded ViT-based models from Mu et al. (2022); Ilharco et al. (2021) and CLIP models fine-tuned on ImageNet, which are only used for evaluation but not fitting the baseline functions. We provide additional details in Appendix B.1.

We use "{Name_of_dataset} models" to denote models trained only on the dataset, e.g., "CIFAR-10 models". And we use "{Name_of_dataset_A}+{Name_of_dataset_B} models" to represent models trained on a combination of two datasets, e.g., "CIFAR-10+ImageNet models".

ID test sets: We focus on image classification. Labeled image classification datasets such as ImageNet can be directly used for evaluating ID accuracy. For datasets that consist of image-text pairs for language-image pre-training without original labels, including YFCC and LAION, we automatically generate classification l